This unit is to read the air temperature 
as well as log GPS (bluefly) and humidity to the SD card

We can get all the selected components working and producing 
same file format as the original one.  And then solder together

Then we need to design a 3D-printed housing for it that rigidly 
attaches to an upright profile

Then take it out and run around mapping warm and cold areas 

(Should work on a car ride)

* Fix the humidity work
* Fix the initial logger plot
* Fix the analog reader rate
* missing date records: Rt0000B90Fd"2017-06-24T13:44:54.600"e000F423Fn00000682f00000682o000F423FA


In [7]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [5]:
%sendtofile --source NewDataLogger_funcs.py


Sent 147 lines (4854 bytes) to NewDataLogger_funcs.py.


In [494]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py
%sendtofile --quiet --source BlueFly_funcs.py
%sendtofile --mkdir --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py /lib/
%sendtofile --quiet --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/OLED_grapher.py
%sendtofile --quiet --source BlueFly_funcs.py


Sent 55 lines (2040 bytes) to SI7021_funcs.py.


In [8]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext, oledshowfattext
from machine import Pin, SPI, UART, ADC
import sdcard, os, time, math, ustruct
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from BlueFly_funcs import uart, SetupGPS, BFVmakesettings, ParseNMEA
from NewDataLogger_funcs import ConnectSDcardFile, ConnectBluefly, readlogSI7021, calchumidtemps, readlogbluefly
from NewDataLogger_funcs import InitBNO055, ConvertQuatToEuler

# SD card uses sck=5, mosi=22, miso=27, cs=23
# OLED uses rst=16, scl=15, sda=4
# uartBlueFly = UART(1, baudrate=57600, rx=13, tx=17)
# Pin36 nickel input, Pin19 bridge resistor
# uart2 = UART(2, baudrate=115200, rx=2, tx=18)

# PRG button used to cycle through modes
butt0 = Pin(0)  
butt0presses = 0
def butt0press(e):
    global butt0presses
    butt0presses += 1
butt0.irq(butt0press, butt0.IRQ_FALLING)

# input value from wire sensor
nickelwire = ADC(Pin(36))
nickelwire.atten(nickelwire.ATTN_11DB)
nickelwire.width(nickelwire.WIDTH_12BIT)
nickelbridge = Pin(19, Pin.OUT)
nickelbridge.value(0)

ConnectBluefly()
sdfile = ConnectSDcardFile()
time.sleep_ms(1000) # show file name

SI7021readfail = 0
try:
    i2c.writeto(0x40, b'\xFE')  # resets chip
except OSError:
    SI7021readfail = -1
    oledshowfattext(["SI7021", "bad"])
    time.sleep_ms(700)

uart2 = None #UART(2, baudrate=115200, rx=2, tx=18)
bnores = bytearray(10)

prevflushstamp = 0
prevoledstamp = 0
prevnickelwireread = -100
prevnickelwirestamp = 0
scrollinit()
gpsfixes = 0
prevbutt0 = butt0.value()
prevdisplaymode = 0
prsrec = 100000
while True:
    tstamp = time.ticks_ms()
    displaymode = butt0presses%5
    
    # should use mag = Pin(19, Pin.OUT) to tune it, but doesn't work
    nickelwireread = nickelwire.read()
    if abs(nickelwireread - prevnickelwireread) >= 4 and tstamp > prevnickelwirestamp + 25:
        sdfile.write("Nt{:08X}s{:06X}\n".format(tstamp, nickelwireread))
        prevnickelwireread = nickelwireread
        prevnickelwirestamp = tstamp
        
    if SI7021readfail != -1:
        try:
            readlogSI7021()
        except OSError:
            SI7021readfail += 1
            
    lbf = readlogbluefly()
    
    if tstamp > prevflushstamp + 2000:
        sdfile.flush()
        prevflushstamp = tstamp

    # all data has been collected.  Now display it
    if type(lbf) == int:
        prsrec = lbf
                
    if displaymode != prevdisplaymode:
        scrollinit()
        k = "display%d"%displaymode
        oledshowfattext(k)
        print(k)
        prevdisplaymode = displaymode
        
    if displaymode == 0:  # display nmea situation
        if type(lbf) == bytes:
            oledshowfattext([lbf[:8], lbf[8:16], lbf[16:24], lbf[24:32]])
            print(lbf)
    elif displaymode == 4:
        if uart2 is None:
            uart2 = UART(2, baudrate=115200, rx=2, tx=18)
            k = InitBNO055(uart2)
            oledshowfattext(["BNO055", k])
            time.sleep_ms(2000)
        uart2.write(b"\xAA\x01\x20\x08")
        time.sleep_ms(20)  # NB delay coded in
        nr = uart2.readinto(bnores)
        if ((bnores[0] == 0xBB) and (bnores[1] == 8) and (nr == 10)):
            k = ustruct.unpack("<hhhh", bnores[2:])
            pitch, roll, orient = ConvertQuatToEuler(*k)
            fbuff.fill(0)
            fbuff.hline(0, max(0, min(63, int(roll+32))), 128, 1)
            fbuff.text("%.1f"%roll, 0, 32, 1)
            fbuff.vline(max(0, min(128, int(pitch*2+64))), 0, 64, 1)
            fbuff.text("%.1f"%pitch, 64, 0, 1)
            fbuff.line(64, 32, 64+int(math.sin(math.radians(orient))*30), 32+int(math.cos(math.radians(orient))*30), 1)
            oledshow()
        else:
            oledshowfattext("BAD BNO")
        
    elif tstamp > prevoledstamp + 200: # scrolling values
        if displaymode == 1:
            addscrollgraph(prsrec-100000, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("prs",0,0,1)
            fbuff.text(str(prsrec),0,8,1)
        elif displaymode == 2:
            addscrollgraph(nickelwireread, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("nickel",0,0,1)
            fbuff.text(str(nickelwireread),0,8,1)
        elif displaymode == 3:
            hs, ts = calchumidtemps()
            addscrollgraph(ts, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("temp",0,0,1)
            fbuff.text(str(nickelwireread),0,8,1)
        else:
            oledshowfattext(["mstamp", str(tstamp)])
        oledshow()
        prevoledstamp = tstamp


Sent 134 lines (4562 bytes) to main.py.


In [25]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7fc8c61ab7b8, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [144]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 1, in <module>\r\n  File "OLED_driver.py", line 5, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [4]:
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext, oledshowfattext
from machine import Pin, SPI, UART, ADC
import sdcard, os, time
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from BlueFly_funcs import uart, SetupGPS, BFVmakesettings, ParseNMEA
from NewDataLogger_funcs import ConnectSDcardFile, ConnectBluefly, readlogSI7021, calchumidtemps, readlogbluefly
from NewDataLogger_funcs import InitBNO055, ConvertQuatToEuler

uart2 = UART(2, baudrate=115200, rx=2, tx=18)


I (22126) uart: ALREADY NULL


In [5]:
from NewDataLogger_funcs import InitBNO055, ConvertQuatToEuler
k = InitBNO055(uart2)
print(k)

Temp31


In [11]:
import ustruct, math
bnores = bytearray(10)
for i in range(1000):
    uart2.write(b"\xAA\x01\x20\x08")
    time.sleep_ms(20)
    nr = uart2.readinto(bnores)
    if not ((bnores[0] == 0xBB) and (bnores[1] == 8) and (nr == 10)):
        continue
    k = ustruct.unpack("<hhhh", bnores[2:])
    pitch, roll, orient = ConvertQuatToEuler(*k)
    fbuff.fill(0)
    fbuff.hline(0, max(0, min(63, int(roll+32))), 128, 1)
    fbuff.vline(max(0, min(128, int(pitch*2+64))), 0, 64, 1)
    fbuff.line(64, 32, 64+int(math.sin(math.radians(orient))*30), 32+int(math.cos(math.radians(orient))*30), 1)
    oledshow()


.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
KeyboardInterrupt: 


In [10]:
print(bnores)


bytearray(b'\xbb\x08&!\t\xff\xac\xf9_6')


In [1395]:
ConnectBluefly()


b'BVL' already good
b'BHV' already good
b'BOF' already good
b'BHT' already good
Change GPS to baud 57600


In [1398]:
for i in range(1000):
    x = uart.readline()
    if x[:3] != b'PRS':
        print(x)


b'$GPGGA,000007.800,,,,,0,0,,,M,,M,,*47\r\n'
b'$GPVTG,0.00,T,,M,0.00,N,0.00,K,N*32\r\n'
b'GGA,000045.200,,,,,0,0,,,M,,M,,*4B\r\n'
b'$GPVTG,0.00,T,,M,0.00,N,0.00,K,N*32\r\n'
b'4\r\n'
b'$GPGGA,000048.400,,,,,0,0,,,M,,M,,*40\r\n'
b'$GPVTG,0.00,T,,M,0.00,N,0.00,K,N*32\r\n'


Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
TypeError: 'NoneType' object is not subscriptable


In [131]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [161]:
import machine
#uart = machine.UART(1, baudrate=115200, rx=17, tx=13)
#uart = machine.UART(2, baudrate=115200, rx=2, tx=18)
uart2.write(b"\xAA\x01\x00\x06")
time.sleep_ms(120)
r = uart2.read()
print(r)


b'\xbb\x06\xa0\xfb2\x0f\x11\x03'


In [173]:
uart2.write(b"\xAA\x01\x00\x06")
time.sleep_ms(120)
r = uart2.read()
print(r)


b'\xbb\x06\xa0\xfb2\x0f\x11\x03'


In [4]:
from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
InitBNO055(uart)


Temperature 31


In [49]:
pitch, roll, orient = BNO055absorientation()
calibstat = BNO055calibstat()
print(pitch, roll, orient, bin(calibstat))



1.507387 -7.663566 89.89901 0b110000


In [38]:
r = bytearray(10)
print(r)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [98]:
import math
def ConvertQuatToEuler(q0, q1, q2, q3):
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/riqsq 
    
    r02 = q0*q2*2 * iqsq
    r13 = q1*q3*2 * iqsq
    sinpitch = r13 - r02

    r01 = q0*q1*2 * iqsq
    r23 = q2*q3*2 * iqsq 
    sinroll = r23 + r01 
     
    r00 = q0*q0*2 * iqsq
    r11 = q1*q1*2 * iqsq
    r03 = q0*q3*2 * iqsq
    r12 = q1*q2*2 * iqsq
    a00=r00 - 1 + r11   
    a01=r12 + r03  
    rads = math.atan2(a00, -a01) 
    northorient = 180 - math.degrees(rads) 
    return math.degrees(math.asin(sinpitch)), math.degrees(math.asin(sinroll)), northorient


In [107]:
import ustruct 
uart.write(b"\xAA\x01\x20\x08")
time.sleep_ms(20)
nr = uart.readinto(r)
if ((r[0] == 0xBB) and (r[1] == 8) and (nr == 10)):
    k = ustruct.unpack("<hhhh", r[2:])
else:
    k = (1,0,0,0)
print(k)
print(ConvertQuatToEuler(*k))


(14524, -361, -489, 7557)
(1.86768, -3.818729, 34.96077)


In [108]:
help(fbuff)

object <FrameBuffer> is of type FrameBuffer
  fill -- <function>
  fill_rect -- <function>
  pixel -- <function>
  hline -- <function>
  vline -- <function>
  rect -- <function>
  line -- <function>
  blit -- <function>
  scroll -- <function>
  text -- <function>


In [130]:
import ustruct 
for i in range(1000):
    uart.write(b"\xAA\x01\x20\x08")
    time.sleep_ms(20)
    nr = uart.readinto(r)
    if not ((r[0] == 0xBB) and (r[1] == 8) and (nr == 10)):
        continue
    k = ustruct.unpack("<hhhh", r[2:])
    pitch, roll, orient = ConvertQuatToEuler(*k)
    fbuff.fill(0)
    fbuff.hline(0, max(0, min(63, int(roll+32))), 128, 1)
    fbuff.vline(max(0, min(128, int(pitch*2+64))), 0, 64, 1)
    fbuff.line(64, 32, 64+int(math.sin(math.radians(orient))*30), 32+int(math.cos(math.radians(orient))*30), 1)
    oledshow()



[missing-OK]K.......
[missing-OK]
[missing-OK]

>

........................................................................................................................................................................................................................................................................................................................................................
[missing-OK]


**[ys] 


[missing-OK]

...

<class 'serial.serialutil.SerialException'>


[missing-OK]


**[ys] 


[missing-OK]

read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)



In [125]:
fbuff.line(10,10, 30, 0, 1)
oledshow()


In [ ]:
void SDloggerBase::logorientA(uint32_t mstamp, int16_t ax, int16_t ay, int16_t az, int16_t gx, int16_t gy, int16_t gz, int16_t qw, int16_t qx, int16_t qy, int16_t qz, uint16_t calibstat) 
{
    int offs = startline('Z', mstamp); 
    sddata[offs++] = 'x'; 
    offs = buff16(offs, ax); 
    sddata[offs++] = 'y'; 
    offs = buff16(offs, ay); 
    sddata[offs++] = 'z'; 
    offs = buff16(offs, az); 
    sddata[offs++] = 'a'; 
    offs = buff16(offs, gx); 
    sddata[offs++] = 'b'; 
    offs = buff16(offs, gy); 
    sddata[offs++] = 'c'; 
    offs = buff16(offs, gz); 
    sddata[offs++] = 'w'; 
    offs = buff16(offs, qw); 
    sddata[offs++] = 'x'; 
    offs = buff16(offs, qx); 
    sddata[offs++] = 'y'; 
    offs = buff16(offs, qy); 
    sddata[offs++] = 'z'; 
    offs = buff16(offs, qz); 
    sddata[offs++] = 's'; 
    offs = buff16(offs, calibstat); 
    writeline(offs); 
}


void SDloggerBase::logorientcalibrations(uint32_t mstamp, uint8_t calibstat, uint8_t* chdata, uint8_t length) 
{
    int offs = startline('Y', mstamp); 
    sddata[offs++] = 'c'; 
    offs = buff8(offs, calibstat); 
    sddata[offs++] = 's'; 
    sddata[offs++] = '"'; 
    for (uint8_t i = 0; i < length; i++) 
        offs = buff8(offs, chdata[i]); 
    sddata[offs++] = '"'; 
    writeline(offs); 
}
